In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import numpy as np 
from scipy.optimize import minimize

from fredapi import Fred    


In [2]:
tickers = ['spy','BND','GLD','QQQ','VTI']

In [3]:
end_date = datetime.today()

In [4]:
start_date = end_date - timedelta(days=365*5)
start_date

datetime.datetime(2020, 3, 10, 16, 4, 15, 124859)

# Downloading Adjusted Close Prices

In [5]:
adj_close = pd.DataFrame()

In [6]:
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    adj_close[ticker] = data['Close']


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
adj_close

,spy,BND,GLD,QQQ,VTI
Date,,,,,
2020-03-10,267.401245,75.312263,154.479996,197.951614,134.729370
2020-03-11,254.365753,73.885864,153.929993,189.329803,127.908112
2020-03-12,230.028809,69.867546,147.789993,171.969925,115.470398
2020-03-13,249.693161,72.816017,143.279999,186.536728,125.980331
2020-03-16,222.370789,73.581413,141.639999,164.191910,111.642693
...,...,...,...,...,...
2025-03-03,583.770020,73.610001,266.739990,497.049988,287.709991
2025-03-04,576.859985,73.400002,269.059998,495.549988,284.119995
2025-03-05,583.059998,73.139999,269.619995,502.010010,287.350006


In [8]:
log_returns = np.log(adj_close/adj_close.shift(1))

In [9]:
log_returns

,spy,BND,GLD,QQQ,VTI
Date,,,,,
2020-03-10,NaN,NaN,NaN,NaN,NaN
2020-03-11,-0.049977,-0.019121,-0.003567,-0.044532,-0.051956
2020-03-12,-0.100569,-0.055920,-0.040706,-0.096171,-0.102298
2020-03-13,0.082028,0.041335,-0.030992,0.081309,0.087112
2020-03-16,-0.115887,0.010457,-0.011512,-0.127592,-0.120822
...,...,...,...,...,...
2025-03-03,-0.017675,0.002299,0.013094,-0.022125,-0.018083
2025-03-04,-0.011908,-0.002857,0.008660,-0.003022,-0.012556
2025-03-05,0.010691,-0.003549,0.002079,0.012952,0.011304


# Calculating the Covariance

In [10]:
cov_matrix = log_returns.cov()*252
cov_matrix

,spy,BND,GLD,QQQ,VTI
spy,0.041027,0.003513,0.005883,0.047239,0.042183
BND,0.003513,0.004936,0.003679,0.004146,0.003709
GLD,0.005883,0.003679,0.023791,0.007743,0.006068
QQQ,0.047239,0.004146,0.007743,0.063200,0.048465
VTI,0.042183,0.003709,0.006068,0.048465,0.043744


# Defining portfolio performance metrics

In [11]:
def standard_deviation (weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

## calculate expected returns

In [12]:
def expected_return (weights, log_returns):
    return np.sum(log_returns.mean()*weights)*252 ## 252 traiding days in the year...

## Calculate the sharp ratio

In [13]:
def sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return(expected_return(weights, log_returns) - risk_free_rate)/standard_deviation(weights, cov_matrix)

# Getting the risk free rate.

In [14]:
fred = Fred(api_key='56573fff95fcef94dc89f1dfef310615')
ten_year_treasury_rate = fred.get_series_latest_release('GS10')/100

In [15]:
risk_free_rate = ten_year_treasury_rate.mean()
risk_free_rate

0.05548030127462341

## Define the function to minimise

In [16]:
def neg_sharpe(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate)

## Set the constraints

In [17]:
constraints = {'type':'eq', 'fun': lambda x: np.sum(x)-1}
bounds = [(0,0.5) for _ in range(len(tickers))]

# Set the initial weights

In [18]:
initial_weights = np.array([1/len(tickers)]*(len(tickers)))
initial_weights

array([0.2, 0.2, 0.2, 0.2, 0.2])

# Optimise the weights to maximise the sharpe ratio 

In [19]:
optimised_returns = minimize(neg_sharpe, initial_weights, args=(log_returns, cov_matrix, risk_free_rate), method = 'SLSQP', constraints=constraints, bounds=bounds)

In [20]:
optimised_returns

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.5723751474020589
       x: [ 1.561e-01  1.299e-16  4.719e-01  3.720e-01  0.000e+00]
     nit: 10
     jac: [-3.571e-01  1.312e-01 -3.576e-01 -3.570e-01 -3.091e-01]
    nfev: 60
    njev: 10

## Getting the optimal results

In [21]:
optimised_weights = optimised_returns.x
optimised_weights

array([1.56097376e-01, 1.29887420e-16, 4.71943835e-01, 3.71958789e-01,
       0.00000000e+00])

## Displaying the analytics of the portfolio

In [22]:
print("Optimal Weights:")
for ticker, weight in zip(tickers, optimised_weights):
    print(f"{ticker}: {weight*100:.4f}%")

optimal_portfolio_return = expected_return(optimised_weights, log_returns)
optimal_portfolio_volatility = standard_deviation(optimised_weights, cov_matrix)
optimal_sharpe_ratio = sharpe_ratio(optimised_weights, log_returns, cov_matrix, risk_free_rate)

print(f"Expected Annual Return: {optimal_portfolio_return:.2f}")
print(f"Expected Volatility: {optimal_portfolio_volatility:.2f}")
print(f"Sharperatio: {optimal_sharpe_ratio:.2f}")

Optimal Weights:
spy: 15.6097%
BND: 0.0000%
GLD: 47.1944%
QQQ: 37.1959%
VTI: 0.0000%
Expected Annual Return: 0.14
Expected Volatility: 0.16
Sharperatio: 0.57
